In [ ]:
%load_ext autoreload
%autoreload 2

import sys; sys.path.insert(0, '..')

In [ ]:
from src.util import sample_from_dict, load_saved_data

In [ ]:
import json
import random
from rich import print
import geopandas as gpd
import pandas as pd

## Load data

In [ ]:
stations = pd.DataFrame(load_saved_data())
datamaps = pd.read_json("../data/datamaps.json")
ne = gpd.read_file("../data/ne_50m_admin_0_countries.geojson")
centers = gpd.read_file("centers.geojson")

In [ ]:
radio_countries = set(stations["country"])
len(radio_countries)

In [ ]:
df = ne.loc[:, ["ADMIN", "ADM0_TLC", "ISO_A2"]]

In [ ]:
# only inlcude countries in radio dataset
df = df[df["ADMIN"].isin(radio_countries)]
df.shape

In [ ]:
# merge with datamaps
df = pd.merge(df, datamaps, left_on="ADM0_TLC", right_on="id")
df.shape

In [ ]:
# merge with centers
df = pd.merge(df, centers, left_on="ISO_A2", right_on="AFF_ISO")
df.shape

In [ ]:
df = df[["ADMIN", "AFF_ISO", "id"]]
df.columns = ["name", "two_code", "three_code",]

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df.to_json("../data/out/source.json", orient="records")

#### Prepare center

In [ ]:
centers = gpd.read_file("../data/centers.geojson")

In [ ]:
centers_main = centers[centers["COUNTRY"] == centers["COUNTRYAFF"]]

In [ ]:
centers_main = centers_main[["COUNTRYAFF", "AFF_ISO", "geometry"]]

In [ ]:
centers_main.to_file("centers.geojson", driver='GeoJSON')

In [ ]:
# ADM0_ISO, ISO_A2
# list(ne.columns)

In [ ]:
len(ne["ISO_A2"].unique())

In [ ]:
centers

## Match data

### Natural Earth

In [ ]:
p = ne["geometry"][0]

In [ ]:
ne.shape

In [ ]:
# seems to be the best unique key?
len(ne["ADM0_TLC"].unique())

In [ ]:
ne["MATCH"] = False
stations["ne_idx"] = -1

In [ ]:
# radio_countries = list(set(stations.country))

In [ ]:
def find_idx(country):
    """Find ne index that corresponds to country name"""
    cols = ["NAME", "ADMIN", "NAME_LONG"]
    for col in cols:
        match_idxs = ne.index[ne[col].str.fullmatch(c)].tolist()
        if match_idxs:
            idx = match_idxs[0]
            ne.loc[idx, ("MATCH")] = True
            return idx
    return -1

In [ ]:
no_match = []
match = []

for i, c in stations.country.iteritems():
    idx = find_idx(c)
    stations.loc[i, ("ne_idx")] = idx
    
    if idx > -1:
        match.append((c, idx))
    else:
        no_match.append(c)    

In [ ]:
# # --- OLD WAY ----

# no_match = []
# match = []

# for c in radio_countries:
#     idx = find(c)
#     if idx:
#         match.append((c, idx))
#     else:
#         no_match.append(c)

In [ ]:
matched = stations[stations["ne_idx"] > -1]

In [ ]:
matched

### Finish up

In [ ]:
code_col = "ADM0_TLC"

In [ ]:
data = pd.DataFrame(matched)
data["code"] = list(ne[code_col].iloc[matched["ne_idx"]])

In [ ]:
data.to_json("../data/out/radio.json", orient='records')

In [ ]:
missing = stations[stations["ne_idx"] == -1]

In [ ]:
# missing

### Fuzzy Matching

In [ ]:
from thefuzz import fuzz
from thefuzz import process

In [ ]:
missing = stations[stations["ne_idx"] == -1]

In [ ]:
ne_still = ne[ne["MATCH"] == False]

In [ ]:
ne_still.shape

In [ ]:
def fuzzy_find(country, col="ADMIN"):
    found = process.extract(
        country, ne_still[col].to_dict(), limit=3, scorer=fuzz.partial_ratio)
    for f in found:
        if f[1] > 80:
            return f
    return None

In [ ]:
fuzzy_find('Guadeloupe')

In [ ]:
# now_found = []
# still_no_match = []

# for c in no_match:
#     match = fuzzy_find(c, "ADMIN")
#     if match:
#         now_found.append((c, match[-1]) + (match[0], match[1]))
#     else:
#         still_no_match.append(c)

In [ ]:
now_found = []
still_no_match = []

for i, c in missing.country.iteritems():
    match = fuzzy_find(c, "ADMIN")
    if match:
        # stations.loc[i, ("ne_idx")] = idx    
        now_found.append((c, match[-1]) + (match[0], match[1]))
    else:
        still_no_match.append(c)

In [ ]:
fuzzy_find("Zimbabwe")

In [ ]:
pd.DataFrame(now_found, columns=["radio", "ne_idx", "ne_admin", "score"])

Everything good except:

- Iraq
- Guyana

In [ ]:
for c in no_match:
    match1 = fuzzy_find(c, "ADMIN")
    match2 = fuzzy_find(c, "NAME_LONG")
    
    if match1 or match2:
        print("-------")
        print(c)
    
    if match1:
        print("ADMIN", match1)
    if match2:
        print("NAME_LONG", match2)
    

In [ ]:
cols = ["NAME", "ADMIN", "NAME_LONG"]
ne_full = set(pd.concat([ne[col] for col in cols]))

In [ ]:
fuzzy = {}
for c in no_match:
    # found = process.extract(c, ne_full, limit=3, scorer=fuzz.partial_ratio)
    found = process.extract(c, ne_full, limit=3)
    fuzzy[c] = found

In [ ]:
for k, v in fuzzy.items():
    print(k, sorted(v, key=lambda x: x[1], reverse=True))

In [ ]:
matches = []

no_match_datamaps_countries = []

for c1 in datamaps_countries:
    match = False
    for c2 in radio_countries:
        if c1.lower() == c2.lower():
            matches.append((c1, c2))
            match = True
    if not match:
        no_match_datamaps_countries.append(c1)

### Load datamaps data

In [ ]:
with open("data/datamaps.json") as f:
    datamaps = json.load(f)

In [ ]:
datamaps_countries = [
    x["properties"]["name"]
    for x in datamaps
]

In [ ]:
len(datamaps_countries)

## Compare

In [ ]:
matches = []

no_match_datamaps_countries = []

for c1 in datamaps_countries:
    match = False
    for c2 in radio_countries:
        if c1.lower() == c2.lower():
            matches.append((c1, c2))
            match = True
    if not match:
        no_match_datamaps_countries.append(c1)

In [ ]:
len(matches)

In [ ]:
no_match_datamaps_countries

In [ ]:
matches = []

no_match_radio_countries = []

for c1 in radio_countries:
    match = False
    for c2 in datamaps_countries:
        if c1.lower() == c2.lower():
            matches.append((c1, c2))
            match = True
    if not match:
        no_match_radio_countries.append(c1)

In [ ]:
no_match_radio_countries